<a href="https://colab.research.google.com/github/mahdi-alalawi/AndroidTutorialForBeginners/blob/master/%D9%85%D8%A7%D9%83%D8%AF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def calculate_macd(data, short_window=12, long_window=26, signal_window=9):
    """حساب مؤشر MACD"""
    data['EMA_short'] = data['close'].ewm(span=short_window, adjust=False).mean()
    data['EMA_long'] = data['close'].ewm(span=long_window, adjust=False).mean()
    data['MACD'] = data['EMA_short'] - data['EMA_long']
    data['Signal_Line'] = data['MACD'].ewm(span=signal_window, adjust=False).mean()
    return data

# استخدام الدالة
data = calculate_macd(data)
ذي

In [ ]:
def calculate_bollinger_bands(data, window=20, num_std_dev=2):
    """حساب بولينجر باندز."""
    try:
        # حساب المتوسط المتحرك
        data['Middle_Band'] = data['close'].rolling(window=window).mean()

        # حساب الانحراف المعياري
        data['Std_Dev'] = data['close'].rolling(window=window).std()

        # حساب الحد العلوي والحد السفلي
        data['Upper_Band'] = data['Middle_Band'] + (data['Std_Dev'] * num_std_dev)
        data['Lower_Band'] = data['Middle_Band'] - (data['Std_Dev'] * num_std_dev)

        return data
    except Exception as e:
        print(f"An error occurred while calculating Bollinger Bands: {e}")
        return data


In [ ]:
data = get_and_clean_gold_prices()
if data is not None:
    data = calculate_bollinger_bands(data)  # إضافة هذه السطر لحساب بولينجر باندز
    build_strategy(data)


In [ ]:
def calculate_stochastic(data, k_window=14, d_window=3):
    """حساب Stochastic Oscillator"""
    low_min = data['low'].rolling(window=k_window).min()
    high_max = data['high'].rolling(window=k_window).max()
    data['%K'] = 100 * (data['close'] - low_min) / (high_max - low_min)
    data['%D'] = data['%K'].rolling(window=d_window).mean()
    return data


In [ ]:
def set_break_even(symbol):
    """تعيين وقف الخسارة إلى مستويات مختلفة بناءً على تقدم السعر نحو الهدف."""
    positions = mt5.positions_get(symbol=symbol)

    if positions:
        for position in positions:
            entry_price = position.price_open
            take_profit = position.tp

            distance_to_tp = take_profit - entry_price

            current_price = mt5.symbol_info_tick(symbol).bid

            fifty_percent_target = entry_price + (0.36 * distance_to_tp)
            seventy_five_percent_target = entry_price + (0.72 * distance_to_tp)
            ninety_percent_target = entry_price + (0.84 * distance_to_tp)

            if current_price >= fifty_percent_target:
                new_stop_loss = entry_price + (0.12 * distance_to_tp)
                update_stop_loss(position, new_stop_loss)

            if current_price >= seventy_five_percent_target:
                new_stop_loss = entry_price + (0.24 * distance_to_tp)
                update_stop_loss(position, new_stop_loss)

            if current_price >= ninety_percent_target:
                new_stop_loss = entry_price + (0.48 * distance_to_tp)
                update_stop_loss(position, new_stop_loss)


In [ ]:
def set_break_even(symbol):
    """تعيين وقف الخسارة إلى مستويات مختلفة بناءً على تقدم السعر نحو الهدف،
    مع عدم تحريك الستوب لوز إذا كان الجديد أقل من القديم."""
    positions = mt5.positions_get(symbol=symbol)

    if positions:
        for position in positions:
            entry_price = position.price_open
            take_profit = position.tp
            current_stop_loss = position.sl  # الحصول على قيمة الستوب لوز الحالية

            distance_to_tp = take_profit - entry_price

            current_price = mt5.symbol_info_tick(symbol).bid

            fifty_percent_target = entry_price + (0.36 * distance_to_tp)
            seventy_five_percent_target = entry_price + (0.72 * distance_to_tp)
            ninety_percent_target = entry_price + (0.84 * distance_to_tp)

            new_stop_loss = current_stop_loss  # افتراض أن الستوب لوز لن يتغير

            if current_price >= fifty_percent_target:
                new_stop_loss = entry_price + (0.12 * distance_to_tp)

            if current_price >= seventy_five_percent_target:
                new_stop_loss = entry_price + (0.24 * distance_to_tp)

            if current_price >= ninety_percent_target:
                new_stop_loss = entry_price + (0.48 * distance_to_tp)

            # مقارنة الستوب لوز الجديد بالقديم
            if new_stop_loss > current_stop_loss:
                update_stop_loss(position, new_stop_loss)

In [ ]:
import pandas as pd

# تعريف دالة حساب ADX
def calculate_adx(data, period=14):
    """حساب مؤشر ADX."""
    data['High_Change'] = data['high'].diff()
    data['Low_Change'] = data['low'].diff()

    data['+DM'] = (data['High_Change'] > data['Low_Change']) & (data['High_Change'] > 0) * data['High_Change']
    data['-DM'] = (data['Low_Change'] > data['High_Change']) & (data['Low_Change'] > 0) * data['Low_Change']

    data['TR'] = data[['high', 'low', 'close']].max(axis=1) - data[['high', 'low', 'close']].min(axis=1)
    data['ATR'] = data['TR'].rolling(window=period).mean()

    data['+DI'] = 100 * (data['+DM'].rolling(window=period).sum() / data['ATR'])
    data['-DI'] = 100 * (data['-DM'].rolling(window=period).sum() / data['ATR'])

    data['DX'] = 100 * (abs(data['+DI'] - data['-DI']) / (data['+DI'] + data['-DI']))
    data['ADX'] = data['DX'].rolling(window=period).mean()

    return data[['ADX']]

# استخدم الدالة مع بياناتك الخاصة
# تأكد من أن لديك DataFrame بالبيانات المطلوبة (high, low, close)
# adx_data = calculate_adx(your_dataframe)

# طباعة النتائج
# print(adx_data)


In [ ]:
import time

def check_for_high_impact_news():
    """تحقق من وجود أخبار قوية قادمة."""
    # يمكنك استخدام مكتبة مثل requests لجلب الأخبار من موقع مثل Investing.com
    # أو يمكنك استخدام API إذا كان متاحًا.
    # هنا مثال بسيط:
    url = "https://www.investing.com/economic-calendar/"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # ابحث عن الأخبار ذات التأثير العالي
    high_impact_news = soup.select('div.high-impact-news')  # هذا افتراض
    return len(high_impact_news) > 0

# تنفيذ الاستراتيجية بشكل مستمر
if __name__ == "__main__":
    while True:
        data = get_and_clean_gold_prices()
        if data is not None:
            build_strategy(data)

        # تحقق من وجود أخبار قوية
        if check_for_high_impact_news():
            time.sleep(30)  # الانتظار لمدة 30 ثانية قبل التكرار
        else:
            time.sleep(60)  # الانتظار لمدة دقيقة قبل التكرار


In [ ]:
def set_break_even(symbol):
    """تعيين وقف الخسارة إلى مستويات مختلفة بناءً على تقدم السعر نحو الهدف،
    مع عدم تحريك الستوب لوز إذا كان الجديد أقل من القديم."""
    positions = mt5.positions_get(symbol=symbol)

    if positions:
        for position in positions:
            entry_price = position.price_open
            take_profit = position.tp
            current_stop_loss = position.sl  # الحصول على قيمة الستوب لوز الحالية

            distance_to_tp = take_profit - entry_price

            current_price = mt5.symbol_info_tick(symbol).bid

            fifty_percent_target = entry_price + (0.36 * distance_to_tp)
            seventy_five_percent_target = entry_price + (0.63 * distance_to_tp)
            ninety_percent_target = entry_price + (0.84 * distance_to_tp)

            new_stop_loss = current_stop_loss  # افتراض أن الستوب لوز لن يتغير

            if current_price >= fifty_percent_target:
                new_stop_loss = entry_price + (0.12 * distance_to_tp)

            if current_price >= seventy_five_percent_target:
                new_stop_loss = entry_price + (0.24 * distance_to_tp)

            if current_price >= ninety_percent_target:
                new_stop_loss = entry_price + (0.48 * distance_to_tp)

            # مقارنة الستوب لوز الجديد بالقديم
            if new_stop_loss > current_stop_loss:
                update_stop_loss(position, new_stop_loss)


In [ ]:
def set_break_even(symbol):
    """تعيين وقف الخسارة إلى مستويات مختلفة بناءً على أقرب سعر وصل إليه السعر نحو الهدف بعد سعر افتتاح الصفقة."""
    positions = mt5.positions_get(symbol=symbol)

    if positions:
        for position in positions:
            entry_price = position.price_open
            take_profit = position.tp
            current_stop_loss = position.sl  # الحصول على قيمة الستوب لوز الحالية

            # الحصول على أسعار التاريخية بعد فتح الصفقة
            historical_prices = get_historical_prices(symbol, entry_price)

            # تحديد أقرب سعر وصل إليه السعر نحو تيك بروفت
            closest_price = max([price for price in historical_prices if price >= entry_price], default=entry_price)

            distance_to_tp = take_profit - entry_price

            fifty_percent_target = entry_price + (0.36 * distance_to_tp)
            seventy_five_percent_target = entry_price + (0.63 * distance_to_tp)
            ninety_percent_target = entry_price + (0.84 * distance_to_tp)

            new_stop_loss = current_stop_loss  # افتراض أن الستوب لوز لن يتغير

            if closest_price >= fifty_percent_target:
                new_stop_loss = entry_price + (0.12 * distance_to_tp)

            if closest_price >= seventy_five_percent_target:
                new_stop_loss = entry_price + (0.24 * distance_to_tp)

            if closest_price >= ninety_percent_target:
                new_stop_loss = entry_price + (0.48 * distance_to_tp)

            # مقارنة الستوب لوز الجديد بالقديم
            if new_stop_loss > current_stop_loss:
                update_stop_loss(position, new_stop_loss)

def get_historical_prices(symbol, entry_price):
    """استرجاع الأسعار التاريخية بعد سعر افتتاح الصفقة."""
    # يمكنك تعديل هذه الدالة لتناسب احتياجاتك، مثل استخدام `copy_rates_from_pos`
    rates = mt5.copy_rates_from_pos(symbol, mt5.TIMEFRAME_M15, 0, 1000)
    prices = [rate['close'] for rate in rates if rate['time'] >= entry_price]
    return prices


In [ ]:
def set_break_even(symbol):
    """تعيين وقف الخسارة إلى أقرب مستوى وصل إليه السعر بعد سعر افتتاح الصفقة."""
    positions = mt5.positions_get(symbol=symbol)

    if positions:
        for position in positions:
            entry_price = position.price_open
            take_profit = position.tp
            current_stop_loss = position.sl  # الحصول على قيمة الستوب لوز الحالية

            # الحصول على الأسعار التاريخية بعد سعر افتتاح الصفقة
            rates = mt5.copy_rates_from_pos(symbol, mt5.TIMEFRAME_M15, 0, 1000)
            df = pd.DataFrame(rates)
            df['time'] = pd.to_datetime(df['time'], unit='s')
            df.set_index('time', inplace=True)

            # تحديد أقرب سعر وصل إليه السعر بعد سعر افتتاح الصفقة
            highest_price_after_entry = df[df.index > entry_price].high.max()

            # حساب المسافة إلى جني الأرباح
            distance_to_tp = take_profit - entry_price

            # تحديد مستويات جديدة لوقف الخسارة
            new_stop_loss = current_stop_loss  # افتراض أن الستوب لوز لن يتغير

            if highest_price_after_entry >= entry_price + (0.36 * distance_to_tp):
                new_stop_loss = entry_price + (0.12 * distance_to_tp)

            if highest_price_after_entry >= entry_price + (0.63 * distance_to_tp):
                new_stop_loss = entry_price + (0.24 * distance_to_tp)

            if highest_price_after_entry >= entry_price + (0.84 * distance_to_tp):
                new_stop_loss = entry_price + (0.48 * distance_to_tp)

            # مقارنة الستوب لوز الجديد بالقديم
            if new_stop_loss > current_stop_loss:
                update_stop_loss(position, new_stop_loss)
